In [1]:
import numpy as np
import tensorflow as tf
import input_data

In [2]:
mnist = input_data.read_data_sets("tf_mnist", one_hot=True)

Extracting tf_mnist\train-images-idx3-ubyte.gz
Extracting tf_mnist\train-labels-idx1-ubyte.gz
Extracting tf_mnist\t10k-images-idx3-ubyte.gz
Extracting tf_mnist\t10k-labels-idx1-ubyte.gz


In [3]:
learning_rate = 1e-2
n_train_steps = 15
batch_size = 100
display_step = 1

In [4]:
n_h1 = 256
n_h2 = 256
n_input = 784
n_classes = 10

In [5]:
def multilayer_perception(x, weights, biases):
    h1 = tf.nn.relu( tf.matmul(x, weights['h1']) + biases['h1'] )
    h2 = tf.nn.relu( tf.matmul(h1, weights['h2'] + biases['h2']) )
    output = tf.matmul( h2, weights['out'] + biases['out'] )
    return output

In [6]:
graph = tf.Graph()
with graph.as_default():
    weights = {
        'h1': tf.Variable(tf.random_normal([n_input, n_h1])),
        'h2': tf.Variable(tf.random_normal([n_h1, n_h2])),
        'out': tf.Variable(tf.random_normal([n_h2, n_classes]))
    }
    biases = {
        'h1': tf.Variable(tf.zeros([n_h1])),
        'h2': tf.Variable(tf.zeros([n_h2])),
        'out': tf.Variable(tf.zeros([n_classes]))
    }

    Xtr = tf.placeholder(tf.float32, [None, n_input])
    Ytr = tf.placeholder(tf.float32, [None, n_classes])

    Yact = multilayer_perception(Xtr, weights, biases)

    cost = tf.reduce_mean( tf.nn.softmax_cross_entropy_with_logits(
        logits = Yact, labels = Ytr))
    tf_train = tf.train.AdamOptimizer(learning_rate).minimize(cost)
    
    correct_pred = tf.equal ( tf.argmax(Yact, 1), tf.argmax(Ytr, 1))
    train_acc = tf.reduce_mean( tf.cast(correct_pred, tf.float32) )

In [7]:
total_batch = int(mnist.train.num_examples/batch_size)
with tf.Session(graph=graph) as sess:
    tf.global_variables_initializer().run()
    for step in range(n_train_steps):
        avg_cost = 0.
        avg_train_acc = 0.
        for i in range(total_batch):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, c_, train_acc_ = sess.run([tf_train, cost, train_acc], feed_dict={
                Xtr: batch_x, Ytr: batch_y })
            avg_cost += float(c_)/total_batch
            avg_train_acc += float(train_acc_)/total_batch
        if (step+1) % display_step == 0:
            print ("Step: %4d, cost: %.4f, train_acc: %.4f"% 
                   (step+1, avg_cost, avg_train_acc))
    
    print ("Optimization finished")
    
    test_acc = train_acc.eval( feed_dict = {
        Xtr: mnist.test.images, Ytr: mnist.test.labels })
    print ("Test Accuracy: %.4f"% (test_acc))

Step:    1, cost: 51.9743, train_acc: 0.8659
Step:    2, cost: 7.9366, train_acc: 0.9360
Step:    3, cost: 4.2065, train_acc: 0.9531
Step:    4, cost: 2.6413, train_acc: 0.9628
Step:    5, cost: 2.0798, train_acc: 0.9678
Step:    6, cost: 1.6323, train_acc: 0.9718
Step:    7, cost: 1.6164, train_acc: 0.9706
Step:    8, cost: 1.3064, train_acc: 0.9735
Step:    9, cost: 1.2482, train_acc: 0.9737
Step:   10, cost: 1.0475, train_acc: 0.9754
Step:   11, cost: 0.8534, train_acc: 0.9764
Step:   12, cost: 0.6673, train_acc: 0.9779
Step:   13, cost: 0.6789, train_acc: 0.9756
Step:   14, cost: 0.4657, train_acc: 0.9787
Step:   15, cost: 0.4619, train_acc: 0.9781
Optimization finished
Test Accuracy: 0.9515
